In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [30]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        allcodelist = ("U001", "U179", "U180") + codeList
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [31]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 1565, 거래소 1562, 코스닥 1416
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (1565 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 290,728,383,400,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 60,496,996,531,500
삼성전자우 상장주식수: 822,886,700, 시가총액 33,697,210,365,000
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 31,560,705,000,000
NAVER 상장주식수: 164,263,395, 시가총액 28,417,567,335,000
셀트리온 상장주식수: 128,346,619, 시가총액 26,824,443,371,000
LG화학 상장주식수: 70,592,343, 시가총액 20,930,629,699,500
LG생활건강 상장주식수: 15,618,197, 시가총액 19,101,054,931,000
현대차 상장주식수: 213,668,187, 시가총액 18,973,735,005,600
삼성물산 상장주식수: 189,690,043, 시가총액 16,616,847,766,800
삼성SDI 상장주식수: 68,764,530, 시가총액 16,503,487,200,000
현대모비스 상장주식수: 95,054,694, 시가총액 16,111,770,633,000
SK텔레콤 상장주식수: 80,745,711, 시가총액 15,382,057,945,500
엔씨소프트

In [32]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [33]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    
    MarketTotal.append([name, item[0], listed, markettot])

In [34]:
import pandas as pd

In [35]:
format(100000000000, ",")

'100,000,000,000'

In [36]:
format(1000000000000, ",")

'1,000,000,000,000'

In [37]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [38]:
# pdMarketTotal.to_csv("../data/KOSPI_LIST.csv", index=None)

In [39]:
# pdMarketTotal = pdMarketTotal.loc[(
    
#     (pdMarketTotal[0].apply(lambda x: x.find("KODEX") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("TIGER") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("KBSTAR") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("HANARO") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ARIRANG") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("KINDEX") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("KOSEF") < 0)) &
#     (pdMarketTotal[0].apply(lambd`a x: x.find("SMART") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("TREX") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("OTM") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ATM") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ETN") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ETF") < 0)) 
    
# )]

In [40]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization".split(",")

In [41]:
pdMarketTotal.loc[pdMarketTotal.Code == "U001"]

,Name,Code,Stock,capitalization
1562,종합주가지수,U001,0,0.0


In [42]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 200선물인버스2X"]

,Name,Code,Stock,capitalization
115,KODEX 200선물인버스2X,A252670,184800000,1.415568e+12


In [43]:
# pdMarketTotal_200.to_csv("../data/KOSPI200_LIST.csv", index=None)

In [44]:
old = pd.read_csv("../data/KOSPI200_LIST.csv")

KODEX = pdMarketTotal.loc[(pdMarketTotal.Name.apply(lambda x: "KODEX" in x))]

out = pdMarketTotal.loc[(
    (pdMarketTotal.Name.isin(old.Name)) |
    (pdMarketTotal.Name.isin(KODEX.Name)) |
    (pdMarketTotal.Name.isin(pdMarketTotal[:100].Name)) |
    (pdMarketTotal.Stock == 0)
)]

In [45]:
out

,Name,Code,Stock,capitalization
0,삼성전자,A005930,5969782,2.907284e+14
1,SK하이닉스,A000660,728002365,6.049700e+13
2,삼성전자우,A005935,822886700,3.369721e+13
3,삼성바이오로직스,A207940,66165000,3.156070e+13
4,NAVER,A035420,164263395,2.841757e+13
...,...,...,...,...
1533,KODEX MSCI EM선물(H),A291890,450000,3.039750e+09
1555,KODEX 운송,A140710,800000,2.104000e+09
1562,종합주가지수,U001,0,0.000000e+00
1563,KOSPI200선물지수,U179,0,0.000000e+00


In [46]:
out.to_csv("../data/KOSPI200_LIST.csv", index=None)